<a href="https://colab.research.google.com/github/Qaiserfarooq285/Labtasks/blob/main/paperwith4000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Downloading Dataset**

In [ ]:
import os
import json
with open('/content/kaggle.json') as f:
    kaggle_json = json.load(f)
    os.environ['KAGGLE_USERNAME'] = kaggle_json['username']
    os.environ['KAGGLE_KEY'] = kaggle_json['key']

In [ ]:
!kaggle datasets download tirendazacademy/fifa-world-cup-2022-tweets

Dataset URL: https://www.kaggle.com/datasets/tirendazacademy/fifa-world-cup-2022-tweets
License(s): CC0-1.0
fifa-world-cup-2022-tweets.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip fifa-world-cup-2022-tweets.zip

Archive:  fifa-world-cup-2022-tweets.zip
replace fifa_world_cup_2022_tweets.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!pip install -U "tensorflow-text==2.13.*"

In [ ]:
!pip install --quiet tensorflow_text
import re
import nltk
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt_tab')
stopwords.words('english')
exclude = string.punctuation

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


# **Reading Data**

In [ ]:
tempdf = pd.read_csv('/content/fifa_world_cup_2022_tweets.csv')
df = tempdf.iloc[:4000]

In [ ]:
df.drop("Unnamed: 0", axis = 1, inplace= True)

<ipython-input-7-8d80297914a7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop("Unnamed: 0", axis = 1, inplace= True)


In [ ]:
df.shape

(4000, 5)

In [ ]:
df.head()

,Date Created,Number of Likes,Source of Tweet,Tweet,Sentiment
0,2022-11-20 23:59:21+00:00,4,Twitter Web App,What are we drinking today @TucanTribe \n@MadB...,neutral
1,2022-11-20 23:59:01+00:00,3,Twitter for iPhone,Amazing @CanadaSoccerEN #WorldCup2022 launch ...,positive
2,2022-11-20 23:58:41+00:00,1,Twitter for iPhone,Worth reading while watching #WorldCup2022 htt...,positive
3,2022-11-20 23:58:33+00:00,1,Twitter Web App,Golden Maknae shinning bright\n\nhttps://t.co/...,positive
4,2022-11-20 23:58:28+00:00,0,Twitter for Android,"If the BBC cares so much about human rights, h...",negative


In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize # Import word_tokenize

def preprocess(text):
    text = re.sub(r'(^|\s)@(\w+)', '', text)
    text = re.sub(r'\bhttps?://\S+\b', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()

    tokens = word_tokenize(text) # Now word_tokenize is accessible
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    text = ' '.join(tokens)
    return text

In [ ]:
df['Tweet'] = df['Tweet'].apply(preprocess)

<ipython-input-11-41eb3d697dbc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Tweet'] = df['Tweet'].apply(preprocess)


In [26]:
word_freq = df.Tweet.str.split().explode().value_counts().reset_index()
word_freq.columns = ["Word", "Frequency"]
word_freq['Frequency'] = word_freq.Frequency.astype(int)

word_freq

,Word,Frequency
0,worldcup,4706
1,qatar,1953
2,world,937
3,fifaworldcup,873
4,cup,835
...,...,...
8268,binancenft,1
8269,ronaldonft,1
8270,crnft,1
8271,thebinancenft,1


In [22]:
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [ ]:
# Map sentiment values to numerical labels
sentiment_mapping = {'positive': 0, 'negative': 1, 'neutral': 2}
df['sentiment_label'] = df['Sentiment'].map(sentiment_mapping)  # Assuming 'Sentiment' is your target column

# X and y are derived from the DataFrame
X = df['Tweet']
y = df['sentiment_label']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

<ipython-input-13-8cf9ca007091>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment_label'] = df['Sentiment'].map(sentiment_mapping)  # Assuming 'Sentiment' is your target column


In [ ]:
# Define BERT model for fine-tuning
preprocessor = hub.KerasLayer("https://kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-preprocess/versions/3")
encoder = hub.KerasLayer("https://www.kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-l-12-h-768-a-12/versions/4", trainable=True)

# Input layer
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
encoder_inputs = preprocessor(text_input)
outputs = encoder(encoder_inputs)

# Pooled output
pooled_output = outputs["pooled_output"]

# Dropout and classification layers
drop_out = tf.keras.layers.Dropout(0.3, name='dropout')(pooled_output)  # Increased dropout for regularization
output = tf.keras.layers.Dense(3, activation='softmax', name='classifier')(drop_out)  # 3 classes for multi-class

# Build the model
model = tf.keras.Model(inputs=[text_input], outputs=[output])

In [ ]:
# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)
batch_size = 16
epochs = 3
history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=epochs,
    batch_size=batch_size,
    callbacks=[early_stopping]
)


Epoch 1/3
180/180 [==============================] - 4089s 23s/step - loss: 0.8787 - accuracy: 0.5931 - val_loss: 0.5862 - val_accuracy: 0.7375
Epoch 2/3
180/180 [==============================] - 4042s 22s/step - loss: 0.5582 - accuracy: 0.7823 - val_loss: 0.6393 - val_accuracy: 0.7625


In [29]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import numpy as np
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")



12/25 [=============>................] - ETA: 4:23

KeyboardInterrupt: 

In [ ]:
rf_model = RandomForestClassifier()
rf_model.fit(train_embeddings, y_train)
y_pred = rf_model.predict(test_embeddings)
print (accuracy_score(y_test,y_pred))
print (confusion_matrix(y_test,y_pred))

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(train_embeddings, y_train)
rf_model.fit(train_embeddings, y_train)
y_pred = rf_model.predict(test_embeddings)
print (accuracy_score(y_test,y_pred))